In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import pypulse as pulsar
import matplotlib.pyplot as plt     # para graficar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [2]:
#----------------------------------------------------------------------------
# Partimos de un conjunto de observaciones mezcladas con diferentes S/N
#----------------------------------------------------------------------------

ant = 'A2'                          # antena usada (A1 o A2)                    

# creamos una lista con todos los archivos .pfd de observaciones

timing_folder = './' + str(ant) + '_SN/'
pfd_files = glob.glob(timing_folder+'*pfd')         # lista de archivos .pfd

print(len(pfd_files))

148


In [1]:
# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])
    
# Save all PSRFITS files
psrfits_files = glob.glob(timing_folder+'*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))
print(len(observations))

NameError: name 'pfd_files' is not defined

In [ ]:
# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    # First crunch in time and frequency
    observation.fscrunch()
    observation.tscrunch()
    
    # Now get array of the best profile
    tmp_singlepulse = observation.getSinglePulses(windowsize=256)
    
    # Align and normalize
    tmp_singlepulse.center_align()
    tmp_singlepulse.normalize()
    
    # Save aligned and normalize in each observation
    single_pulses.append(tmp_singlepulse)
    
# Save single pulse object
sp_observations = dict(zip(psrfits_files, single_pulses)
print(len(sp_observations))

In [ ]:
# y nos quedamos con los .pfds de S/N >= SN_min

files_50 = []
files_100 = []
files_150 = []
files_200 = []

for key in observations:
    
    sn = sp_observations[key].getSN()
    temp = key.replace("./A2_SN/", "")
    file = temp.replace(".fits", "pfd")
        
    if sn > 50:
        files_50.append(file)
        
    if sn > 100:
        files_100.append(file)
        
    if sn > 150:
        files_150.append(file)
        
    if sn > 200:
        files_200.append(file)
        
print(len(files_50))
print(len(files_100))
print(len(files_150))
print(len(files_200))

In [ ]:
files_50.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')
files_100.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')
files_150.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')
files_200.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')

print(len(files_50))
print(len(files_100))
print(len(files_150))
print(len(files_200))

In [3]:
#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

sn_min = 50

template = "template_1024_A2.std.sm"  # nombre del template
timing_file = "timing_" + str(sn_min) + ".tim"       # nombre del archivo .tim

print("template = " + str(template) )
print("timing file = " + str(timing_file) )

# primero, generamos un string que contenga todos los archivos .pfd

archivos = ''
for pfd in files_50:
    archivos = archivos + pfd + ' '
        
# ahora llamamos a pat para generar el archivo .tim

subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, cwd=timing_folder, shell = True)



#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

residue_file = "residues_" + str(sn_min) + ".dat"   # nombre del archivo de los residuos

subprocess.check_call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file, cwd=timing_folder, shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

RMS = np.genfromtxt ( str(ant) + '_' + str(nbin) + '/' + residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

print(str(sn_min) + ' ' + str(RMS) )

template = template_1024_A2.std.sm
timing file = timing_50.tim


NameError: name 'pfd_files' is not defined